In [1]:
import numpy as np
from datetime import datetime

np.random.seed(0)

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [3]:
class Diabetes:
    def __init__(self, i_node, h_node, o_node, lr):
        self.learning_rate = lr
        
        self.W2 = np.random.rand(i_node, h_node)
        self.b2 = np.random.rand(1)
        
        self.W3 = np.random.rand(h_node, o_node)
        self.b3 = np.random.rand(1)
        
        print("========== Diabetes Object is created. ==========\n")
        
    def feed_forward(self, x_data, t_data):
        delta = 1e-7
        
        z2 = np.dot(self.x_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        return -np.sum((self.t_data*np.log10(a3 + delta)) + ((1 - self.t_data)*np.log10((1 - a3) + delta)))

    def loss_val(self):
        delta = 1e-7
        
        z2 = np.dot(self.x_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        return -np.sum((self.t_data*np.log10(a3 + delta)) + ((1 - self.t_data)*np.log10((1 - a3) + delta)))
    
    def train(self, x_data, t_data):
        self.x_data = x_data
        self.t_data = t_data
        
        f = lambda x : self.feed_forward(self.x_data, self.t_data)
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
    def predict(self, test_data):
        z2 = np.dot(test_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        if a3 >= 0.5:
            result = 1
        else:
            result = 0
    
        return a3, result
    
    def accuracy(self, test_xdata, test_tdata):
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(test_xdata)
        
        return accuracy_val

In [5]:
try:
    data = np.loadtxt('./diabetes.csv', delimiter=',', dtype=np.float32)
    training_data = data[ :int(len(data)*0.7),  : ]
    test_data = data[int(len(data)*0.7): ,  : ]
    
    print("training_data.shape = ", training_data.shape, ", test_data.shape = ", test_data.shape)
    
except Exception as err:
    print(str(err))

training_data.shape =  (531, 9) , test_data.shape =  (228, 9)


In [8]:
i_node = 8
h_node = 10
o_node = 1
lr = 1e-2
epoch = 20

obj1 = Diabetes(i_node, h_node, o_node, lr)

print("Neural Network Learning using Numerical Derivative...\n")

start_time = datetime.now()

for step in range(epoch):
    for index in range(len(training_data)):
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        obj1.train(input_data, target_data)
        
    if step % (int)(0.05*epoch) == 0:
        print("epochs = ", step, " : ", "loss value = ", obj1.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

========== Diabetes Object is created. ==========

Neural Network Learning using Numerical Derivative...

epochs =  0  :  loss value =  0.08143367631610597
epochs =  1  :  loss value =  0.14423119845343793
epochs =  2  :  loss value =  0.1729996275365351
epochs =  3  :  loss value =  0.18463585868308233
epochs =  4  :  loss value =  0.1894949206430241
epochs =  5  :  loss value =  0.1918111872056071
epochs =  6  :  loss value =  0.19317930610313055
epochs =  7  :  loss value =  0.19419208781528466
epochs =  8  :  loss value =  0.19507210704304628
epochs =  9  :  loss value =  0.19590472848765442
epochs =  10  :  loss value =  0.1967238332183171
epochs =  11  :  loss value =  0.19754373457397548
epochs =  12  :  loss value =  0.19837129056833988
epochs =  13  :  loss value =  0.19921060961461118
epochs =  14  :  loss value =  0.20006491716215893
epochs =  15  :  loss value =  0.20093729327269166
epochs =  16  :  loss value =  0.20183093790489856
epochs =  17  :  loss value =  0.20274922

In [9]:
try:
    input_data = test_data[:, 0:-1]
    target_data = test_data[:, [-1]]

    accuracy_ret = obj1.accuracy(input_data, target_data)
    
    print('Accuracy => ', accuracy_ret)
    
except Exception as err:
    print(str(err))

Accuracy =>  0.6578947368421053
